Load s3fs ... skip if we already have it:

In [1]:
# pip install s3fs

Enumerate 2019 Yellow Cab files

In [1]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
trips = fs.ls('nyc-tlc/trip data', detail=True)
files = ['s3://' + trip['Key'] for trip in trips if ('yellow_tripdata_2019' in trip['Key'])]
files

['s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-02.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-03.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-04.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-05.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-06.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-07.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-08.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-09.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-10.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-11.csv',
 's3://nyc-tlc/trip data/yellow_tripdata_2019-12.csv']

Grab just the first 2, as that's all we can fit locally

In [3]:
import pandas as pd

In [4]:
%%time


ldf = pd.concat([pd.read_csv(f) for f in files[:2]], ignore_index = True)

CPU times: user 25.5 s, sys: 5.16 s, total: 30.6 s
Wall time: 1min 3s


(about 60 seconds to load)

In [5]:
floatcols = [c for c in ldf.columns if ldf[c].dtype == 'float64']
floatcols

['trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

2 files 45 sec end to end
3 files ~60 sec to load, then crashes

Drop rows w missing values

In [6]:
clean = ldf[floatcols].dropna().values
clean

array([[ 1.2,  6.5,  0. , ...,  0.3,  7.3,  0. ],
       [ 0.6,  4. ,  0. , ...,  0.3,  6.3,  0. ],
       [ 2.2, 10. ,  0. , ...,  0.3, 11.8,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ]])

In [11]:
clean.shape

(9831189, 9)

First 2 files contain ~10 million records

Run SVD to get the singular values

In [7]:
from scipy import linalg

In [15]:
%%timeit

U, s, V = linalg.svd(clean, full_matrices=False)

s

1.69 s ± 6.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In my test on BlazingSQL, this takes about 1.7 seconds and uses 2/4 cores